# BNN-BN
This notebook demonstrates the implementation of the BNN-BN Paper
# Steps to binarize the model
- Load Dataset and DataLoader
- Create the teacher model to be binarized
- Load the configuration YAML file
- create `BNNBN object` and pass the student model, dataloader, configuration
- Binarize the model by using `compress_model` method

In [1]:
import sys
import os
import shutil

sys.path.append("../../../")

import torch
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory
from trailmet.algorithms.binarize.utils import Lighting
from trailmet.algorithms.binarize.BNNBN import BNNBN
import torchvision

import yaml

/opt/conda/envs/py117/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
root_dir = "./"
import os

with open(os.path.join(root_dir, "bnnbn_cifar100.yaml"), "r") as stream:
    kwargs = yaml.safe_load(stream)
kwargs

{'DATASET': 'c100',
 'arch': 'reactnet-A',
 'binary_w': True,
 'bn_type': 'none',
 'batch_size': 256,
 'workers': 4,
 'valid_size': 0.1,
 'num_train': 0,
 'num_classes': 100,
 'insize': 32,
 'loss_type': 'ce',
 'teacher': 'resnet34',
 'teacher_weight': '',
 'label_smooth': 0.1,
 'pretrained': '',
 'resume': False,
 'save': './saved_weights',
 'epoch': 5,
 'agc': True,
 'clip_value': 0.04,
 'weight_decay': 0,
 'learning_rate': 0.001,
 'wandb': True}

In [4]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    print("> SEEDING DONE")


set_seed(1024)

> SEEDING DONE


# Defining the Model 

In [5]:
model = ModelsFactory.create_model(name=kwargs["arch"], **kwargs)

* Model = reactnet-A
* Binarize both activation and weights
* without BN


# Augmentation, Dataset and DataLoaders

In [6]:
# Augmentation
crop_scale = 0.08
lighting_param = 0.1
train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(32, scale=(crop_scale, 1.0)),
        Lighting(lighting_param),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

valid_transform = transforms.Compose(
    [transforms.Resize(36), transforms.CenterCrop(32), transforms.ToTensor()]
)
test_transform = valid_transform

input_transforms = {
    "train": train_transform,
    "val": valid_transform,
    "test": test_transform,
}

target_transforms = {"train": None, "val": None, "test": None}

# Create Dataset
cifar100_dataset = DatasetFactory.create_dataset(
    name="CIFAR100",
    root="./",
    split_types=["train", "val", "test"],
    val_fraction=0.15,
    transform=input_transforms,
    target_transform=target_transforms,
)
# Define DataLoaders
train_loader100 = torch.utils.data.DataLoader(
    cifar100_dataset["train"],
    batch_size=kwargs["batch_size"],
    sampler=cifar100_dataset["train_sampler"],
    num_workers=kwargs["workers"],
)
val_loader100 = torch.utils.data.DataLoader(
    cifar100_dataset["val"],
    batch_size=kwargs["batch_size"],
    sampler=cifar100_dataset["val_sampler"],
    num_workers=kwargs["workers"],
)
test_loader100 = torch.utils.data.DataLoader(
    cifar100_dataset["test"],
    batch_size=kwargs["batch_size"],
    sampler=cifar100_dataset["test_sampler"],
    num_workers=kwargs["workers"],
)

dataloaders = {"train": train_loader100, "val": val_loader100, "test": test_loader100}

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# ReActNet Object

In [7]:
a = BNNBN(model, dataloaders, **kwargs)

wandb: Currently logged in as: animesh-007. Use `wandb login --relogin` to force relogin


# Binarizing the model

In [8]:
fin = a.compress_model()

Student Model:reactnet(
  (feature): ModuleList(
    (0): firstconv3x3(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    )
    (1): BasicBlock(
      (move11): LearnableBias()
      (binary_3x3): HardBinaryConv()
      (move12): LearnableBias()
      (prelu1): PReLU(num_parameters=32)
      (move13): LearnableBias()
      (move21): LearnableBias()
      (binary_pw_down1): HardBinaryConv()
      (binary_pw_down2): HardBinaryConv()
      (move22): LearnableBias()
      (prelu2): PReLU(num_parameters=64)
      (move23): LearnableBias()
      (binary_activation): BinaryActivation()
      (pooling): AvgPool2d(kernel_size=2, stride=2, padding=0)
    )
    (2): BasicBlock(
      (move11): LearnableBias()
      (binary_3x3): HardBinaryConv()
      (move12): LearnableBias()
      (prelu1): PReLU(num_parameters=64)
      (move13): LearnableBias()
      (move21): LearnableBias()
      (binary_pw_down1): HardBinaryConv()
      (binary_pw_down2): HardB

Training in BNNBN Epoch [X] (X / X Steps) (batch time=X.Xs) (data time=X.Xs) (loss=X.X) (top1=X.X) (top5=X.X):   0%|| 0/167 [00:00<?, ?it/s]/opt/conda/envs/py117/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Training in BNNBN Epoch [0] (167 / 167 Steps) (batch time=0.20986s) (data time=0.00181s) (loss=4.11096) (top1=0.00000) (top5=25.00000): 100%|| 167/167 [00:14<00:00, 11.82it/s]
Validating Epoch [0] (30 / 30 Steps) (batch time=0.16224s) (loss=4.35254) (top1=6.57895) (top5=17.10526): 100%|| 30/30 [00:01

 * acc@1 2.760 acc@5 13.360


Training in BNNBN Epoch [1] (167 / 167 Steps) (batch time=0.06216s) (data time=0.00189s) (loss=3.29017) (top1=50.00000) (top5=50.00000): 100%|| 167/167 [00:12<00:00, 13.78it/s]
Validating Epoch [1] (30 / 30 Steps) (batch time=0.01842s) (loss=4.05580) (top1=7.89474) (top5=22.36842): 100%|| 30/30 [00:01<00:00, 26.43it/s]


 * acc@1 6.507 acc@5 23.773


...

 * acc@1 11.240 acc@5 32.013


Training in BNNBN Epoch [3] (167 / 167 Steps) (batch time=0.06308s) (data time=0.00186s) (loss=3.39987) (top1=25.00000) (top5=25.00000): 100%|| 167/167 [00:11<00:00, 13.99it/s]
Validating Epoch [3] (30 / 30 Steps) (batch time=0.01755s) (loss=3.55981) (top1=11.84210) (top5=40.78947): 100%|| 30/30 [00:01<00:00, 25.59it/s]


 * acc@1 14.827 acc@5 39.453


Training in BNNBN Epoch [4] (167 / 167 Steps) (batch time=0.06289s) (data time=0.00193s) (loss=3.15846) (top1=0.00000) (top5=25.00000): 100%|| 167/167 [00:12<00:00, 13.61it/s] 
Validating Epoch [4] (30 / 30 Steps) (batch time=0.01780s) (loss=3.48275) (top1=17.10526) (top5=43.42105): 100%|| 30/30 [00:01<00:00, 25.31it/s]


 * acc@1 14.840 acc@5 39.453
Total training time = 0.03377752906746335 hours
Best acc = 14.84000015258789
